## SpaCy Evaluation - 02/1 - qualitative Analyse

Es wird folgendes betrachtet: 
- True Postives / Anzahl, Beispiele
- False Postives  / Anzahl, Beispiele, Kategorien
- False Negatives  / Anzahl


---
### True Positives

In [2]:
import pandas as pd
import json
from IPython.display import display


with open("../../data/NER/spacy/results_de_core_news_lg.json", encoding="utf-8") as f:
    pred_data = json.load(f)

with open("../../data/data_annotated.json", encoding="utf-8") as f:
    gold_data = json.load(f)

# 2. Golddaten nach file_name indexieren
gold_index = {entry["file_name"]: entry for entry in gold_data}
relevant_label = "LOC"

# 3. TP sammeln + Goldvergleich einbauen
true_positives_all = []

for pred in pred_data:
    file_name = pred["file_name"]
    text = pred["text"]
    tps = [e for e in pred.get("true_positives", []) if e["label"] == relevant_label]

    gold_entry = gold_index.get(file_name, {})
    gold_locs = [
        {
            "start": e["start"],
            "end": e["end"],
            "label": e["label"],
            "text": text[e["start"]:e["end"]]
        }
        for e in gold_entry.get("entities", [])
        if e["label"] == relevant_label
    ]

    for tp in tps:
        start, end = tp["start"], tp["end"]
        tp_entry = {
            "file_name": file_name,
            "text_fragment": text[start:end],
            "start": start,
            "end": end,
            "label": tp["label"],
            "full_text": text,
            "gold_locs": gold_locs  
        }
        true_positives_all.append(tp_entry)


df_tp = pd.DataFrame(true_positives_all)


df_tp = df_tp[["file_name", "text_fragment", "start", "end", "gold_locs"]]

pd.set_option('display.max_colwidth', None)


df_tp.head(20)

,file_name,text_fragment,start,end,gold_locs
0,0002.jpg,Großer Garten,147,160,"[{'start': 138, 'end': 145, 'label': 'LOC', 'text': 'Dresden'}, {'start': 147, 'end': 160, 'label': 'LOC', 'text': 'Großer Garten'}]"
1,0002.jpg,Dresden,138,145,"[{'start': 138, 'end': 145, 'label': 'LOC', 'text': 'Dresden'}, {'start': 147, 'end': 160, 'label': 'LOC', 'text': 'Großer Garten'}]"
2,0003.jpg,Berlin-Moabit,146,159,"[{'start': 122, 'end': 129, 'label': 'LOC', 'text': 'Mahnmal'}, {'start': 130, 'end': 144, 'label': 'LOC', 'text': 'Levetzowstraße'}, {'start': 146, 'end': 159, 'label': 'LOC', 'text': 'Berlin-Moabit'}]"
3,0007.jpg,Sonntagstraße,66,79,"[{'start': 50, 'end': 58, 'label': 'LOC', 'text': 'Ostkreuz'}, {'start': 66, 'end': 79, 'label': 'LOC', 'text': 'Sonntagstraße'}]"
4,0007.jpg,Ostkreuz,50,58,"[{'start': 50, 'end': 58, 'label': 'LOC', 'text': 'Ostkreuz'}, {'start': 66, 'end': 79, 'label': 'LOC', 'text': 'Sonntagstraße'}]"
5,0011.jpg,Riesa,38,43,"[{'start': 38, 'end': 43, 'label': 'LOC', 'text': 'Riesa'}]"
6,0012.jpg,Berlin,61,67,"[{'start': 61, 'end': 67, 'label': 'LOC', 'text': 'Berlin'}, {'start': 154, 'end': 160, 'label': 'LOC', 'text': 'Berlin'}, {'start': 273, 'end': 279, 'label': 'LOC', 'text': 'Berlin'}]"
7,0012.jpg,Berlin,154,160,"[{'start': 61, 'end': 67, 'label': 'LOC', 'text': 'Berlin'}, {'start': 154, 'end': 160, 'label': 'LOC', 'text': 'Berlin'}, {'start': 273, 'end': 279, 'label': 'LOC', 'text': 'Berlin'}]"
8,0013.jpg,BERLIN,0,6,"[{'start': 0, 'end': 6, 'label': 'LOC', 'text': 'BERLIN'}, {'start': 12, 'end': 24, 'label': 'LOC', 'text': 'ORANIENPLATZ'}]"
9,0015.jpg,ESSEN,48,53,"[{'start': 48, 'end': 53, 'label': 'LOC', 'text': 'ESSEN'}, {'start': 54, 'end': 57, 'label': 'LOC', 'text': 'HBF'}]"


In [4]:
df_tp.sample(40)[["file_name", "text_fragment", "start", "end", "gold_locs"]]

,file_name,text_fragment,start,end,gold_locs
59,0152.jpg,A100,45,49,"[{'start': 45, 'end': 49, 'label': 'LOC', 'text': 'A100'}, {'start': 84, 'end': 109, 'label': 'LOC', 'text': 'Auffahrt Tempelhofer Damm'}]"
28,0071.jpg,Siegessäule,101,112,"[{'start': 77, 'end': 94, 'label': 'LOC', 'text': 'Brandenburger Tor'}, {'start': 101, 'end': 112, 'label': 'LOC', 'text': 'Siegessäule'}]"
52,0140.jpg,Schöneweide,68,79,"[{'start': 61, 'end': 64, 'label': 'LOC', 'text': 'HTW'}, {'start': 68, 'end': 79, 'label': 'LOC', 'text': 'Schöneweide'}]"
8,0013.jpg,BERLIN,0,6,"[{'start': 0, 'end': 6, 'label': 'LOC', 'text': 'BERLIN'}, {'start': 12, 'end': 24, 'label': 'LOC', 'text': 'ORANIENPLATZ'}]"
23,0061.jpg,Schloss Bellevue,116,132,"[{'start': 116, 'end': 132, 'label': 'LOC', 'text': 'Schloss Bellevue'}]"
9,0015.jpg,ESSEN,48,53,"[{'start': 48, 'end': 53, 'label': 'LOC', 'text': 'ESSEN'}, {'start': 54, 'end': 57, 'label': 'LOC', 'text': 'HBF'}]"
29,0071.jpg,Brandenburger Tor,77,94,"[{'start': 77, 'end': 94, 'label': 'LOC', 'text': 'Brandenburger Tor'}, {'start': 101, 'end': 112, 'label': 'LOC', 'text': 'Siegessäule'}]"
46,0128.jpg,Berlin,204,210,"[{'start': 113, 'end': 140, 'label': 'LOC', 'text': 'Kulturquartier silent green'}, {'start': 204, 'end': 210, 'label': 'LOC', 'text': 'Berlin'}]"
85,0219.jpg,Leopoldplatz,36,48,"[{'start': 36, 'end': 48, 'label': 'LOC', 'text': 'Leopoldplatz'}, {'start': 59, 'end': 68, 'label': 'LOC', 'text': 'Mauerpark'}]"
69,0191.jpg,Oranienplatz,40,52,"[{'start': 40, 'end': 52, 'label': 'LOC', 'text': 'Oranienplatz'}]"


In [8]:
import json

with open("../../data/NER/spacy/results_de_core_news_lg.json", encoding="utf-8") as f:
    pred_data = json.load(f)



false_negatives = [
    fn for entry in pred_data
    for fn in entry.get("true_positives", [])
    if fn.get("label") == "LOC"
]


total_nf = len(false_negatives)

# Ausgabe
print(f" Gesamtzahl True Positives (LOC): {total_nf}")

 Gesamtzahl True Positives (LOC): 95


---
### False Positives

In [5]:
import pandas as pd
import json
from IPython.display import display


with open("../../data/NER/spacy/results_de_core_news_lg.json", encoding="utf-8") as f:
    pred_data = json.load(f)

with open("../../data/data_annotated.json", encoding="utf-8") as f:
    gold_data = json.load(f)

# 2. Golddaten nach file_name indexieren
gold_index = {entry["file_name"]: entry for entry in gold_data}
relevant_label = "LOC"

# 3. FP sammeln + Goldvergleich einbauen
false_positives_all = []

for pred in pred_data:
    file_name = pred["file_name"]
    text = pred["text"]
    fps = [e for e in pred.get("false_positives", []) if e["label"] == relevant_label]

    gold_entry = gold_index.get(file_name, {})
    gold_locs = [
        {
            "start": e["start"],
            "end": e["end"],
            "label": e["label"],
            "text": text[e["start"]:e["end"]]
        }
        for e in gold_entry.get("entities", [])
        if e["label"] == relevant_label
    ]

    for fp in fps:
        start, end = fp["start"], fp["end"]
        fp_entry = {
            "file_name": file_name,
            "text_fragment": text[start:end],
            "start": start,
            "end": end,
            "label": fp["label"],
            "full_text": text,
            "gold_locs": gold_locs  # Liste aller korrekten LOCs für Vergleich
        }
        false_positives_all.append(fp_entry)


df_fp = pd.DataFrame(false_positives_all)


df_fp = df_fp[["file_name", "text_fragment", "start", "end", "gold_locs"]]

pd.set_option('display.max_colwidth', None)


df_fp.head(20)

,file_name,text_fragment,start,end,gold_locs
0,0003.jpg,Mahnmal Levetzowstraße,122,144,"[{'start': 122, 'end': 129, 'label': 'LOC', 'text': 'Mahnmal'}, {'start': 130, 'end': 144, 'label': 'LOC', 'text': 'Levetzowstraße'}, {'start': 146, 'end': 159, 'label': 'LOC', 'text': 'Berlin-Moabit'}]"
1,0004.jpg,Islandstraße,143,155,"[{'start': 117, 'end': 141, 'label': 'LOC', 'text': 'Grundschule Islandstraße'}, {'start': 143, 'end': 158, 'label': 'LOC', 'text': 'Islandstraße 25'}]"
2,0004.jpg,Islandstraße,129,141,"[{'start': 117, 'end': 141, 'label': 'LOC', 'text': 'Grundschule Islandstraße'}, {'start': 143, 'end': 158, 'label': 'LOC', 'text': 'Islandstraße 25'}]"
3,0006.jpg,STADT,17,22,[]
4,0009.jpg,Burggasse,74,83,"[{'start': 58, 'end': 72, 'label': 'LOC', 'text': 'Ateliertheater'}, {'start': 74, 'end': 86, 'label': 'LOC', 'text': 'Burggasse 71'}, {'start': 88, 'end': 97, 'label': 'LOC', 'text': '1070 Wien'}]"
5,0009.jpg,Wien,93,97,"[{'start': 58, 'end': 72, 'label': 'LOC', 'text': 'Ateliertheater'}, {'start': 74, 'end': 86, 'label': 'LOC', 'text': 'Burggasse 71'}, {'start': 88, 'end': 97, 'label': 'LOC', 'text': '1070 Wien'}]"
6,0013.jpg,Kaloyan Velkov,184,198,"[{'start': 0, 'end': 6, 'label': 'LOC', 'text': 'BERLIN'}, {'start': 12, 'end': 24, 'label': 'LOC', 'text': 'ORANIENPLATZ'}]"
7,0017.jpg,Bahnsteig S-Pankow,175,193,"[{'start': 98, 'end': 120, 'label': 'LOC', 'text': 'Sowjetisches Ehrenmahl'}, {'start': 121, 'end': 125, 'label': 'LOC', 'text': 'Buch'}, {'start': 175, 'end': 184, 'label': 'LOC', 'text': 'Bahnsteig'}, {'start': 185, 'end': 193, 'label': 'LOC', 'text': 'S-Pankow'}]"
8,0025.jpg,Hauptbahnhof +++ SACHSEN,115,139,"[{'start': 115, 'end': 127, 'label': 'LOC', 'text': 'Hauptbahnhof'}]"
9,0027.jpg,A100,0,4,"[{'start': 90, 'end': 98, 'label': 'LOC', 'text': 'Ostkreuz'}, {'start': 114, 'end': 127, 'label': 'LOC', 'text': 'Invalidenpark'}]"


In [10]:
df_fp.sample(40)[["file_name", "text_fragment", "start", "end", "gold_locs"]]

,file_name,text_fragment,start,end,gold_locs
12,0029.jpg,Ostkreuz,69,77,"[{'start': 0, 'end': 19, 'label': 'LOC', 'text': 'Zukunft am Ostkreuz'}, {'start': 21, 'end': 35, 'label': 'LOC', 'text': 'Laskerstraße 5'}, {'start': 37, 'end': 49, 'label': 'LOC', 'text': '10245 Berlin'}, {'start': 198, 'end': 209, 'label': 'LOC', 'text': 'Rudolfplatz'}]"
54,0101.jpg,Straßenbezug,544,556,"[{'start': 61, 'end': 81, 'label': 'LOC', 'text': 'Alt-Hohenschönhausen'}, {'start': 82, 'end': 100, 'label': 'LOC', 'text': 'Suermondtstraße 46'}, {'start': 102, 'end': 114, 'label': 'LOC', 'text': '13053 Berlin'}]"
33,0067.jpeg,Gaza,59,63,"[{'start': 130, 'end': 141, 'label': 'LOC', 'text': 'Café Wostok'}, {'start': 143, 'end': 158, 'label': 'LOC', 'text': 'Weitlingstr. 97'}, {'start': 159, 'end': 164, 'label': 'LOC', 'text': '10317'}]"
178,0241.jpg,NICHTOHNEUNS NICHTOHNEUNS,110,135,"[{'start': 43, 'end': 63, 'label': 'LOC', 'text': 'Rosa-Luxemburg-Platz'}]"
74,0110.jpg,STADT,7,12,"[{'start': 57, 'end': 71, 'label': 'LOC', 'text': 'MARKGRAFENDAMM'}]"
39,0075.jpg,Blank,77,82,"[{'start': 85, 'end': 100, 'label': 'LOC', 'text': 'S-Bahn Ostkreuz'}]"
154,0208.jpg,Falkensee,221,230,"[{'start': 85, 'end': 102, 'label': 'LOC', 'text': 'Falkensee Bahnhof'}, {'start': 129, 'end': 137, 'label': 'LOC', 'text': 'Ostkreuz'}]"
48,0093.jpg,BERLIN,51,57,"[{'start': 0, 'end': 8, 'label': 'LOC', 'text': 'KIEZKLUB'}, {'start': 9, 'end': 16, 'label': 'LOC', 'text': 'Rathaus'}, {'start': 17, 'end': 29, 'label': 'LOC', 'text': 'Johannisthal'}, {'start': 30, 'end': 43, 'label': 'LOC', 'text': 'Sterndamm 102'}, {'start': 45, 'end': 57, 'label': 'LOC', 'text': '12487 BERLIN'}, {'start': 61, 'end': 69, 'label': 'LOC', 'text': 'Innenhof'}]"
66,0101.jpg,parterre,501,509,"[{'start': 61, 'end': 81, 'label': 'LOC', 'text': 'Alt-Hohenschönhausen'}, {'start': 82, 'end': 100, 'label': 'LOC', 'text': 'Suermondtstraße 46'}, {'start': 102, 'end': 114, 'label': 'LOC', 'text': '13053 Berlin'}]"
72,0106.jpg,Rosa-Luxemburg-Platz,212,232,"[{'start': 630, 'end': 650, 'label': 'LOC', 'text': 'Karl Liebknecht Haus'}, {'start': 778, 'end': 800, 'label': 'LOC', 'text': 'Rosa-Luxemburg Platz 1'}, {'start': 802, 'end': 813, 'label': 'LOC', 'text': '1312 Berlin'}]"


In [18]:
import json

with open("../../data/NER/spacy/results_de_core_news_lg.json", encoding="utf-8") as f:
    pred_data = json.load(f)



false_negatives = [
    fn for entry in pred_data
    for fn in entry.get("false_positives", [])
    if fn.get("label") == "LOC"
]


total_nf = len(false_negatives)

# Ausgabe
print(f" Gesamtzahl False Positives (LOC): {total_nf}")

 Gesamtzahl False Positives (LOC): 179


#### Die False Positives im Json wurden manuell gelabelt und in folgende Kategorien eingeteilt:

**loc_valid: true** (LOC korrekt aber nicht relevant für die Veranstaltung)  
**partial_match: true** (LOC nur partiell erkannt wurde, z.B.: Hausnummer fehlt)    
**merged_match: true** (LOC besteht aus mehreren einzelnen LOC Entitäten des Goldstandards)  
**split_entity: true** (LOC einzeln annotiert im Goldstandard aber zusammen)  
**correct_label: ...** (wird nur gesetzt, wenn eigentlich eine andere sinnvolle Entitäte erkannt worden ist, z.B. DATE)    
**extended: true** (korrektes LOC wurde um Bestandteile erweitert, die nicht zu LOC gehören)



In [21]:
import json

with open("../../data/NER/spacy/results_de_core_news_lg.json", encoding="utf-8") as f:
    pred_data = json.load(f)


# Alle False Positives mit label=LOC extrahieren
false_positives = [
    fp for entry in pred_data
    for fp in entry.get("false_positives", [])
    if fp.get("label") == "LOC"
]

# 1. Gesamtanzahl False Positives
total_fp = len(false_positives)

# 2. loc_valid: true
valid_loc = sum(1 for fp in false_positives if fp.get("loc_valid") is True)

# 3. partial_match: true
partial = sum(1 for fp in false_positives if fp.get("partial_match") is True)

# 4. merged_gold_entities: true
merged = sum(1 for fp in false_positives if fp.get("merged_match") is True)

# 5. split_entity: true
split = sum(1 for fp in false_positives if fp.get("split_entity") is True)

# 6. extended: true
extended = sum(1 for fp in false_positives if fp.get("extended") is True)

# 7. correct_label gesetzt
correct_label_set = sum(1 for fp in false_positives if "correct_label" in fp and fp["correct_label"] not in [None, "", "NONE"])

# 8. keine Zusatzinfo gesetzt (nichts von oben)
no_annotation = sum(
    1 for fp in false_positives
    if not any(fp.get(key) for key in ["loc_valid", "partial_match", "merged_match", "split_entity", "correct_label", "extended"])
)

# Ausgabe
print(f"1. Gesamtzahl False Positives (LOC): {total_fp}")
print(f"2. loc_valid = true               : {valid_loc}")
print(f"3. partial_match = true           : {partial}")
print(f"4. merged_matched = true          : {merged}")
print(f"5. split_entity = true            : {split}")
print(f"6. extended = true                : {extended}")
print(f"7. correct_label gesetzt          : {correct_label_set}")
print(f"8. keine Zusatzinfo gesetzt       : {no_annotation}")

1. Gesamtzahl False Positives (LOC): 179
2. loc_valid = true               : 65
3. partial_match = true           : 57
4. merged_matched = true          : 11
5. split_entity = true            : 1
6. extended = true                : 8
7. correct_label gesetzt          : 13
8. keine Zusatzinfo gesetzt       : 24


#### Anzeige von extended

In [13]:
extended = [
    fp for entry in pred_data
    for fp in entry.get("false_positives", [])
    if fp.get("extended") is True 
]

import pandas as pd
df_extended = pd.DataFrame(extended)

# Optional: relevante Spalten anzeigen
df_extended = df_extended[["text", "start", "end", "label"]]


df_extended

,text,start,end,label
0,Berlin-Reinickendorf Komm,108,133,LOC
1,Schönhauser Allee Wann,52,74,LOC
2,Hauptplatz 18:00,84,100,LOC
3,Schöneweide Bring,110,127,LOC
4,10.+11.08.24 Fürstenfeldbruck,72,101,LOC
5,Breitenbachplatz Sonntag,76,100,LOC
6,Gegenprotest Neptunbrunnen,203,229,LOC
7,Forckenbeckplatz Berlin,32,55,LOC


#### Anzeige von correct_label (Label korrigiert)

In [15]:
corrected = [
    fp for entry in pred_data
    for fp in entry.get("false_positives", [])
    if "correct_label" in fp
]


import pandas as pd
df_corrected = pd.DataFrame(corrected)

# Optional: relevante Spalten anzeigen
df_corrected = df_corrected[["text", "start", "end", "label", "correct_label"]]


df_corrected

,text,start,end,label,correct_label
0,Kaloyan Velkov,184,198,LOC,PER
1,Kiezdemo,167,175,LOC,EVENT
2,Schierenberg 60 Leseklubs Stiftung,198,232,LOC,ORG
3,DEMO,45,49,LOC,DATE
4,SOMMERFEST,70,80,LOC,EVENT
5,22.12.1878,157,167,LOC,DATE
6,Wiener Kultursommer,347,366,LOC,EVENT
7,KUNDGEBUNG,69,79,LOC,EVENT
8,Kundgebung,186,196,LOC,EVENT
9,Ori Raz,240,247,LOC,PER


#### Anzeige von LOC  - keine sinnvolle Entitäten

In [20]:

extra_keys = {"loc_valid", "partial_match", "merged_match", "split_entity", "correct_label", "extended"}


no_info = [
    fp for entry in pred_data
    for fp in entry.get("false_positives", [])
    if extra_keys.isdisjoint(fp.keys()) 
]

import pandas as pd
df_no_info = pd.DataFrame(no_info)

# Optional: relevante Spalten anzeigen
df_no_info = df_no_info[["text", "start", "end", "label"]]


df_no_info


,text,start,end,label
0,Krügelredner,46,58,LOC
1,h48,0,3,LOC
2,Späteinlass,207,218,LOC
3,B Tekno,238,245,LOC
4,Gabber,247,253,LOC
5,B!,0,2,LOC
6,Blank,77,82,LOC
7,BREAKDOWN,202,211,LOC
8,Straßenbezug,544,556,LOC
9,BEFREIUNG.UND.WIDERSTAND,184,208,LOC


---
### False Negatives

In [7]:
import json

with open("../../data/NER/spacy/results_de_core_news_lg.json", encoding="utf-8") as f:
    pred_data = json.load(f)



false_negatives = [
    fn for entry in pred_data
    for fn in entry.get("false_negatives", [])
    if fn.get("label") == "LOC"
]


total_nf = len(false_negatives)

# Ausgabe
print(f" Gesamtzahl False Negatives (LOC): {total_nf}")

 Gesamtzahl False Negatives (LOC): 320
